# **Study on the Ideal Behaviour, on the highway, when another vehicle is merging into it - Change Lanes. Access of the influence on the ego vehicle**

##### This study aims to determine the optimal strategy for the ego vehicle when another vehicle is merging into its highway. The only variables under consideration is the lane changing reward `lane_changing_reward`. The goal is to find the **effect of changing lanes** when approaching a certain distance from the merging point, ensuring both safety and traffic efficiency.

### **Imports**

In [1]:
import imageio
import cv2
from IPython.display import Video
import gymnasium as gym
from matplotlib import pyplot as plt
import pprint
from highway_env.envs import MergeEnv
from highway_env import utils
import time
import numpy as np
from stable_baselines3 import PPO
import os
%matplotlib inline

### **Custom environment**

With the ego-vehicle on the highway and a merging vehicle

**Fixed Rewards**
- high speed = 1
- lane change = -5
- right lane = 3
- changing lanes = 5 (at 5 meters from the merging vehicle)

In [2]:
class CustomMergeEnv(MergeEnv):
    def _make_vehicles(self) -> None:
        
        road = self.road
        # Ego vehicle
        ego_vehicle = self.action_type.vehicle_class(
            road, road.network.get_lane(("a", "b", 1)).position(30, 0), speed=30
        )
        road.vehicles.append(ego_vehicle)

        other_vehicles_type = utils.class_from_path(self.config["other_vehicles_type"])

        # Merging vehicle
        merging_v = other_vehicles_type(
            road, road.network.get_lane(("j", "k", 0)).position(110, 0), speed=20
        )
        merging_v.target_speed = 30
        road.vehicles.append(merging_v)
        
        # Set the ego vehicle as the primary vehicle
        self.vehicle = ego_vehicle

    def _reward(self, action: int) -> float:
        """
        Custom reward function combining the original reward with proximity-based
        braking behavior to allow the merging vehicle to merge safely.
        """
        # Get the original reward from the parent class (if it exists)
        reward = super()._reward(action)
        
        ego_vehicle = self.vehicle
        road = self.road

        # Find the merging vehicle (the vehicle in the merging lane)
        merging_vehicle = None
        for vehicle in road.vehicles:
            if vehicle.lane_index == ("j", "k", 0):  # Assuming this is the merging lane
                merging_vehicle = vehicle
                break
        
        # Calculate distance to the merging vehicle
        if not merging_vehicle:
            return reward

        distance= abs(ego_vehicle.position[0] - merging_vehicle.position[0])
        
        if distance < 5 : # If the ego vehicle if getting close to the merging vehicle
            
            lane_changing_reward = self.config.get("lane_changing_reward", 5)
           
        else:
            lane_changing_reward = self.config.get("lane_changing_reward", 0)
        
        reward += lane_changing_reward

        return reward

In [3]:
# Registering the custom environment
gym.envs.registration.register(
    id='CustomMerge-v0',
    entry_point='__main__:CustomMergeEnv',  # Entry point for your custom environment
)

## **Training**

In [4]:
env_change_lanes = CustomMergeEnv(render_mode='rgb_array',config={"high_speed_reward": 1, "lane_change_reward": -5, "right_lane_reward": 3})

In [ ]:
model = PPO('MlpPolicy', env_change_lanes,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            n_steps=2048, 
            batch_size=64, 
            n_epochs=10,  
            gamma=0.8,
            gae_lambda=0.95, 
            clip_range=0.2, 
            verbose=1,
            tensorboard_log="env_change_lanes_ego_influence/")
timesteps = 1000000
model.learn(total_timesteps=timesteps)
model.save("env_change_lanes_ego_influence/model")

## **Testing**

- Average Reward: 7.9466
- Average Steps to Merge: 14.0
- Average Episode Time: 0.31 seconds
- Number of Collisions: 0
- Successful Merges: 200
- Number of Dangerous Driving Episodes (sudden speed changes): 0

In [5]:
# Função para testar o agente e coletar métricas
def evaluate_agent(model, env, num_episodes, speed_threshold_ratio=0.5):
    total_rewards = []
    total_collisions = 0
    successful_merges = 0
    dangerous_driving_episodes = 0
    total_steps_to_merge = []
    total_episode_times = []  # Lista para armazenar os tempos de cada episódio

    # Calcula o threshold com base no intervalo de velocidade
    reward_speed_range = env.config["reward_speed_range"]
    speed_threshold = (reward_speed_range[1] - reward_speed_range[0]) * speed_threshold_ratio
    
    ego_vehicle = env.vehicle
    road = env.road
    
    for vehicle in road.vehicles:
        if vehicle.lane_index == ("j", "k", 0) and vehicle!=ego_vehicle:  # Assuming this is the merging lane
            merging_vehicle = vehicle
            break

    for episode in range(num_episodes):
        start_time = time.time()  # Registra o tempo de início do episódio
        obs, info = env.reset()
        done = False
        episode_reward = 0
        collisions = 0
        dangerous_driving = False
        steps_to_merge = 0
        last_speed = None  # Inicializa a velocidade anterior como None
        changed_lane = False
        

        while not done:
            # O agente escolhe uma ação
            action, _states = model.predict(obs, deterministic=True)

            if action == 0:
                changed_lane = True
            # Executa a ação no ambiente
            obs, reward, terminated, truncated, info = env.step(action)

            episode_reward += reward
            steps_to_merge += 1

            # Verifica a velocidade atual e arredonda para 2 casas decimais
            current_speed = round(info.get('speed', 0), 2)

            # Verifica se houve uma mudança brusca de velocidade
            if last_speed is not None and abs(current_speed - last_speed) > speed_threshold:
                dangerous_driving = True

            last_speed = current_speed  # Atualiza a velocidade anterior

            # Verifica se houve colisão
            if 'crashed' in info and info['crashed']:
                collisions += 1

            # Verifica se o episódio terminou (seja por 'terminated' ou 'truncated')
            done = terminated or truncated

            # Condição de sucesso: sem colisões, o veículo de merging fez o merge corretamente e o ego mudou de faixa
            if not collisions and changed_lane and tuple(merging_vehicle.lane_index[:2]) in {("b", "c"), ("c", "d")}:
                successful_merges += 1
                done = True

        # Registra as métricas do episódio
        total_rewards.append(episode_reward)
        total_collisions += collisions
        total_steps_to_merge.append(steps_to_merge)

        if dangerous_driving:
            dangerous_driving_episodes += 1

        # Calcula o tempo do episódio e adiciona à lista
        episode_time = time.time() - start_time
        total_episode_times.append(episode_time)

    # Cálculos das métricas finais
    avg_reward = np.mean(total_rewards)
    avg_steps_to_merge = np.mean(total_steps_to_merge)
    avg_episode_time = np.mean(total_episode_times)  # Calcula o tempo médio de cada episódio

    # Exibir resultados
    print(f"Average Reward: {avg_reward}")
    print(f"Average Steps to Merge: {avg_steps_to_merge}")
    print(f"Average Episode Time: {avg_episode_time:.2f} seconds")  # Exibe o tempo médio dos episódios
    print(f"Number of Collisions: {total_collisions}")
    print(f"Successful Merges: {successful_merges}")
    print(f"Number of Dangerous Driving Episodes (sudden speed changes): {dangerous_driving_episodes}")

    return {
        "avg_reward": avg_reward,
        "avg_steps_to_merge": avg_steps_to_merge,
        "avg_episode_time": avg_episode_time,  # Adiciona o tempo médio de episódio ao retorno
        "number_collisions": total_collisions,
        "successful_merges": successful_merges,
        "number_dangerous_episodes": dangerous_driving_episodes
    }

In [6]:
# Carregar o modelo treinado e avaliar
model = PPO.load("env_change_lanes_ego_influence/model")

# Avaliar o modelo
results = evaluate_agent(model, env_change_lanes, 200)

c:\Users\Caty\bii\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overTrue
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overTrue
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overTrue
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overF

The results of the study indicate that the ego vehicle performed well in maintaining a balance between **safety** and **efficiency**. The **average reward** for the lane-changing strategy was **7.95**, reflecting a **moderate success** in encouraging the ego vehicle to change lanes while keeping risk levels manageable. This reward suggests that the ego vehicle was able to make reasonable decisions in the merging scenario, although there is still room for improvement to maximize lane-changing effectiveness.

The model required an **average of 14 steps** to merge, with each episode taking an **average of 0.31 seconds**. These consistent values suggest that the ego vehicle was able to effectively manage its response to the merging vehicle, performing the task quickly and efficiently. 

In terms of safety, there were **no collisions** during any of the test episodes, which is a strong indicator that the lane-changing behavior was effective in avoiding accidents. Furthermore, there were **no dangerous driving episodes**, such as sudden speed changes, showing that the ego vehicle’s lane changes were smooth and did not cause any abrupt or unsafe maneuvers. This is crucial for ensuring **traffic safety** and preventing potential hazards on the road.

Perhaps the most significant result of the study is the achievement of **200 successful merges**. This outcome suggests that the lane-changing strategy was not only safe but also effective in facilitating smooth merging with the other vehicle. The success rate of 100% in this regard indicates that the ego vehicle was able to handle the merging scenario with both **efficiency** and **safety**, making the lane changes at the right moments to ensure successful integration with the traffic flow.

In conclusion, the study demonstrates that lane changing at the appropriate moments before a merging point can ensure both **safety** and **traffic efficiency**. The ego vehicle’s ability to achieve **200 successful merges** without collisions or dangerous driving episodes highlights the effectiveness of the lane-changing strategy. However, the **moderate reward** suggests that there may still be opportunities for fine-tuning the approach to further enhance both the merging efficiency and the overall safety of the ego vehicle.

In [9]:
# Load the trained model
model = PPO.load("env_change_lanes_ego_influence/model")

# Initialize the environment and variables for recording
frames = []
obs, info = env_change_lanes.reset()
done = False
step_count = 0
max_steps = 1000

# Resize frame to be divisible by 16 (macro block size for video codecs)
def resize_frame_to_macro_block_size(frame, block_size=16):
    h, w, _ = frame.shape
    new_w = (w // block_size) * block_size
    new_h = (h // block_size) * block_size
    return cv2.resize(frame, (new_w, new_h))

# Run the agent in the environment
while step_count < max_steps and not done:
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env_change_lanes.step(action)
    frame = env_change_lanes.render()

    # Resize the frame to avoid the macro_block_size warning
    resized_frame = resize_frame_to_macro_block_size(frame)
    frames.append(resized_frame)
    
    step_count += 1

# Close the environment
env_change_lanes.close()

# Save the frames as a video
video_filename = "lane_change_ego_influence.mp4"
imageio.mimsave(video_filename, frames, fps=30)
print(f"Video saved as {video_filename}")

c:\Users\Caty\bii\lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overFalse
crashFalse
overTrue
Video saved as lane_change_ego_influence.mp4


In [10]:
# Display the video
video_filename = "lane_change_ego_influence.mp4"
Video(video_filename, embed=True)